## 목표
- 네이버 영화리뷰 데이터 셋을 이용해서 긍정/ 부정 분류기를 만들자
- TF-IDF 방법을 적용해서 토큰화 작업을 해보자 
- konlpy 한국어 형태소 분석기를 설치하고 활용해보자
- 단어별 긍/부정 정보를 시각화 해서 확인해보자

In [84]:
# 1. Java 버전 확인
# cmd 창에서 java - version 확인하기 -> 1.7버전 이상인지 확인해보자
# 만약 java가 없는 경우 설치후 시스템 환경변수 편집까지 해주자

In [85]:
# 2. Jpype 설치
# knolpy는 자바 기반의 라이브러리 => 파이썬 환경에서 원활하게 사용하도록 해주는 별도의 도구가 필요
# !pip install jpype1

In [86]:
# 3. konlypy설치
# konlpy는한국어 형태소 분석기를 모아둔 라이브러리
# !pip install konlpy

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 네이버 데이터 로딩

In [88]:

df_train = pd.read_csv('./data/ratings_train.csv')
df_test = pd.read_csv('./data/ratings_test.csv')

In [89]:
print(df_train.shape)
print(df_test.shape)

(150000, 3)
(50000, 3)


In [90]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [91]:
df_train[df_train.isnull()['document']]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [92]:
df_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [93]:
df_test[df_test.isnull()['document']]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


 - train 데이터와 test 데이터에 document 컬럼에 각각 5개/3개의 결측치가 확인
- 결측치의 형태를 보니 직접적인 리뷰데이터 => 채워줄 수 있는 방법이 없음 => 삭제

In [94]:
# dropna(axis = 0 / 1 , inplace = ) :결측치가 있는 데이터를 삭제하는 함수
# df_train.dropna(axis = 0 , inplace = True)
# df_test.dropna(axis = 0 , inplace = True)

In [95]:
# 결측치 삭제 후 잘 지워 졌는지 확인
df_train.info()
df_train[df_train['document'].isnull()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [96]:
# 결측치 제거 후 확인
df_test.info()
df_train[df_train['document'].isnull()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [97]:
df_train.head()
# id : 리뷰작성한 사용자의 정보
# document : 실제 리뷰 내용
# label : 긍정/ 부정을 표시하는 정답 레이블/ 0 : 긍정 1 : 부정

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [98]:
df_test.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [99]:
text_train = df_train.iloc[ : , 1 ]
y_train = df_train.iloc[ : , 2]
text_test = df_test.iloc[ : , 1]
y_test = df_test.iloc[ : , 2]

In [100]:
print(text_train.shape)
print(y_train.shape)
print(text_test.shape)
print(y_test.shape)


(150000,)
(150000,)
(50000,)
(50000,)


### Bow (가방 만들기)
**Vectorizer**
- vectorizer는 말 그대로 어떠한 대상을 vector화 시켜주는 작업을 수행하는 도구
- 머신러닝에서는 주로 텍스트데이터를 쉽게 분석하기 위해 vector화 시킬때 자주 사용
- 숫자나 벡터값을 input입력값으로 기대하는 여러머신러닝 모델을 실행할때 텍스트나 또 다른 형태의 데이터를 숫자나 벡터값으로 표시할 필요가 있고, 이때 사용하는 것이 vectorizer

** BOW(Bag Of Word/단어 가방) **
- 문서를 벡터로 변환하는 가장 기본적인 방법
- 문서에 있는 모든 단어를 한고셍 모아서 가방을 만들어주고 특정 문서에 어떠한 단어가 있는지 리스트 형태로 순서 - 숫자로 나타내는 방법

In [101]:
# TF-IDF Vectorixer불러오기
from sklearn.feature_extraction.text import TfidfVectorizer

In [102]:
# 테스트용 데이터 가져오기
text_train[ : 3]


0                  아 더빙.. 진짜 짜증나네요 목소리
1    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                    너무재밓었다그래서보는것을추천한다
Name: document, dtype: object

In [103]:
# 객체 생성
tf_idf_vect = TfidfVectorizer()
# 단어사전 구축 : fit()
tf_idf_vect.fit(text_train[ : 3])
# 분리된 문장으로 만든 단어 사전 출력
tf_idf_vect.vocabulary_.keys()


dict_keys(['더빙', '진짜', '짜증나네요', '목소리', '포스터보고', '초딩영화줄', '오버연기조차', '가볍지', '않구나', '너무재밓었다그래서보는것을추천한다'])

- 방금전에 사용해본 TF-IDF vectorizer는 기본적으로 어절단위(문장을 이루는 토막)로 끊기 때문에 한국어 특성상 의미 파악이 어려움

### 한국어 형태소 분석기
**konlpy**-Kkma 사용 예정 : 부르는 이름은 꼬꼬마!, 서울대학교 IDS 연구소에서 개발
- nouns() : 명사만 추출
- morphs() : 형태소만 추출
- pos() : 형태소 추출 + 품사 정보 태그를 부착


In [104]:
# 라이브러리 불러오기
from konlpy.tag import Kkma

In [105]:
# kkma객체 생성
kkma = Kkma()

In [106]:
# 한번 써보자!
text_train[0]


'아 더빙.. 진짜 짜증나네요 목소리'

In [107]:
# 명사만 추출 nouns
kkma.nouns(text_train[0])

['더빙', '목소리']

In [108]:
# TF-IDF kkma연결하기
# 1. 함수 만들기
def myTokenizer(text) : 
    return kkma.nouns(text)

In [109]:
# 객체 생성
tf_idf_vect = TfidfVectorizer(tokenizer=myTokenizer)
# 단어사전 구축 : fit()
tf_idf_vect.fit(text_train[ : 3])
# 분리된 문장으로 만든 단어 사전 출력
tf_idf_vect.vocabulary_.keys()


c:\Users\smhrd\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


dict_keys(['더빙', '목소리', '흠', '포스터', '포스터보고', '보고', '초', '초딩영화줄', '딩', '영화', '줄', '오버', '오버연기', '연기', '재', '재밓', '밓', '추천'])

## 형태소 확인해보기

In [110]:
# kkm.morphs() : 형태소 추출하는 함수
kkma.morphs(text_train[0])

['아', '아', '더빙', '..', '진짜', '짜증나', '네요', '목소리']

In [111]:
kkma.pos(text_train[0])

[('아', 'VV'),
 ('아', 'ECS'),
 ('더빙', 'NNG'),
 ('..', 'SW'),
 ('진짜', 'MAG'),
 ('짜증나', 'VV'),
 ('네요', 'EFN'),
 ('목소리', 'NNG')]

In [113]:
# 품사 태그 종류 출력해보기
kkma.tagset

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [118]:
# 품사정보를 인덱스로 가지는 데이터 프레임 만들어보기
# 조건 : 동사/형용사/ 명사만 추출
data = '먹는다 먹다 이쁘다 아름답다 옷 키보드 마우스 모니터'

# 데이터 프레임 만들기
d = pd.DataFrame(kkma.pos(data),columns=['morphs','tag'])
# 인덱스 지정
d.set_index('tag',inplace = True)
# 동사, 형용사, 명사만 추출해보기
d.loc[d.index.intersection(['VV','VA','NNG'])]

,morphs
tag,
VV,먹
VV,먹
VA,이쁘
VA,아름답
NNG,옷
NNG,키보드
NNG,마우스
NNG,모니터
